# **package_LAB**

In [157]:
import package_LAB
from importlib import reload
package_LAB = reload(package_LAB)

from package_LAB import LL_RT, PID_RT
from package_DBR import SelectPath_RT, Delay_RT

In [158]:
#plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interactive, VBox

## LL_RT()

In [159]:
help(LL_RT)

Help on function LL_RT in module package_LAB:

LL_RT(MV, Kp, TLead, TLag, Ts, PV, PVInit=0, method='EBD')
    The function "FO_RT" needs to be included in a "for or while loop".

    :MV: input vector
    :Kp: process gain
    :TLead: Lead time constant [s]
    :TLag: Lag time constant [s]
    :Ts: sampling period [s]
    :PV: output vector
    :PVInit: (optional: default value is 0)
    :method: discretization method (optional: default value is 'EBD')
        EBD: Euler Backward difference
        EFD: Euler Forward difference
        TRAP: Trapezoidal method

    The function "FO_RT" appends a value to the output vector "PV".
    The appended value is obtained from a recurrent equation that depends on the discretization method.



In [160]:
# Define simulation parameters
TSim = 100
Ts = 0.5
N = int(TSim / Ts) + 1
theta = 5

# Define MV path
MVPath = {0: 0, 5: 1, 50: 2, 80: 3, TSim: 3}

# Initialize lists
t = []
MV = []
MVDelay = []
PV_EBD_1 = []

# Generate time vector and simulate MV and MV_Delay
for i in range(0, N):
    t.append(i * Ts)
    SelectPath_RT(MVPath, t, MV)
    Delay_RT(MV, theta, Ts, MVDelay)


In [161]:
# Create figure
f = go.FigureWidget(make_subplots(rows=2, cols=1, specs = [[{}], [{}]], vertical_spacing = 0.1, row_heights=[0.5, 0.5]))

f.add_trace(go.Scatter(x=t, y=MVDelay, showlegend=False), row=1, col=1)
f.add_trace(go.Scatter(x=t, y=PV_EBD_1, showlegend=False), row=2, col=1)
    

# Add PV_EBD_1 traces for different lead times
def update_TLead(TLead, TLag, Kp):
    
    # Reset list
    PV_EBD_1 = []
    
    # Generate time vector and simulate MV, MV_Delay, and PV_EBD_1
    for i in range(0, N):
        LL_RT(MVDelay[0:i+1], Kp, TLead, TLag, Ts, PV_EBD_1)

    f.data[1].y = PV_EBD_1
    
# Update layout
f['layout'].update(height=800, width=800)
f['layout']['xaxis1'].update(title='Time (s)')
f['layout']['yaxis1'].update(title='MV_Delay')
f['layout']['xaxis2'].update(title='Time (s)')
f['layout']['yaxis2'].update(title='PV_EBD_1')

# Create interactive widget
TLead_slider = interactive(update_TLead, TLead=(0, 50, 1), TLag=(0, 50, 1), Kp=(0, 10, 0.1))

# Display figure and widget
vb = VBox([f, TLead_slider])
vb.layout.align_items = 'center'
vb

    'data': [{'showlegend': False,
              'type': 'scatter',
            …

## PID_RT

In [162]:
help(PID_RT)

Help on function PID_RT in module package_LAB:

PID_RT(SP, PV, Man, MVMan, MVFF, Kc, Ti, Td, alpha, Ts, MVMin, MVMax, MV, MVP, MVI, MVD, E, ManFF=False, PVInit=0, methodI='EBD', methodD='EBD')
    The function "PID_RT" needs to be included in a "for or while loop".

    :SP: SetPoint vector
    :PV: Process Value vector
    :Man: Manual controller mode vector [bool]
    :MVMan: Manual value for MV vector
    :MVFF: FeedForward vector

    :Kc: Controller gain [float]
    :Ti: Integral time constant [s]
    :Td: Derivative time constant [s]
    :alpha: Proportionality factor for Tfd = alpha*Td where Tfd is the derivative filter time constant [s]
    :Ts: Sampling period [s]

    :MVMin: Minimum value for MV (for saturation of MV) [float]
    :MVMax: Maximum value for MV (for saturation of MV) [float]

    :MV: Manipulated Value vector
    :MVP: Proportional part of MV vector
    :MVI: Integral part of MV vector
    :MVD: Derivative part of MV vector
    :E: Control Error vector

    :Ma

In [169]:
# Define simulation parameters
TSim = 300
Ts = 0.5
N = int(TSim / Ts) + 1

# Initialize lists
t = []
SP = []
PV = []
Man = []
MVMan = []
MVFF = []
MV = []
MVp = []
MVi = []
MVd = []
E = []


# Set maximum and minimum MV values
MVmin = 0
MVmax = 100

# Define paths for manual mode
ManPath = {0: 0, 130: True, 170: False} 
MVManPath = {0: 30}

# Define paths for SP and PV
SPPath = {0: 50, 5: 60} 
PVpath = {0: 50}   

MVFFPath = {0: 0, TSim: 0}

for i in range(0, N):
    t.append(i * Ts)
    SelectPath_RT(SPPath, t, SP)
    SelectPath_RT(PVpath, t, PV)
    SelectPath_RT(ManPath, t, Man)
    SelectPath_RT(MVManPath, t, MVMan)
    SelectPath_RT(MVFFPath, t, MVFF)


# Add PV_EBD_1 traces for different TD, Kc and alpha values
def update_PID(Kc, TD, alpha):
        
    # Reset list
    MV = []
    MVp = []
    MVi = []
    MVd = []
    E = []

    TI = TD/0.25
    
    # Generate time vector and simulate MV, MVp, MVi, MVd, and E
    for i in range(0, N):
        PID_RT(SP[0:i+1], PV[0:i+1], Man[0:i+1], MVMan[0:i+1], MVFF[0:i+1], Kc, TI, TD, alpha, Ts, MVmin, MVmax, MV, MVp, MVi, MVd, E)
    
    # Update traces
    fig.data[0].y = SP
    fig.data[1].y = E
    fig.data[2].y = Man
    fig.data[3].y = MV
    fig.data[4].y = MVp
    fig.data[5].y = MVi
    fig.data[6].y = MVd


# Create figure
fig = go.FigureWidget(make_subplots(rows=3, cols=1, specs = [[{}], [{}], [{}]], vertical_spacing = 0.2, row_heights=[0.45, 0.45, 0.1], subplot_titles=("Input and error signals", "MV components")))
fig.add_trace(go.Scatter(x=t, y=SP, name="SP"), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=E, name="E"), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=Man, name="Man"), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=MV, name="MV"), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVp, name="MVp", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVi, name="MVi", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVd, name="MVd", line=dict(dash='dash')), row=2, col=1)


# Update layout
fig['layout'].update(height=800, width=800)
fig['layout']['xaxis1'].update(title='Time (s)')
fig['layout']['xaxis2'].update(title='Time (s)')

# Create interactive widget
TLead_slider = interactive(update_PID, Kc=(0, 5, 0.1), TD=(1, 51, 1), alpha=(0.2, 0.9, 0.1))

# Display figure and widget
vb = VBox([fig, TLead_slider])
vb.layout.align_items = 'center'
vb

    'data': [{'name': 'SP',
              'type': 'scatter',
              'uid'…